<a href="https://colab.research.google.com/github/koaladeschamps/fichiers-perso/blob/main/PySpark_CheatSheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

!pip install -q findspark
import findspark
findspark.init()



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json


df = spark.read.json("/tmp/sample_books.json")

df.printSchema()

--2023-11-08 13:17:34--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2023-11-08 13:17:34 (22.7 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [4]:
#Afficher le tableau
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

In [5]:
#Afficher le nombre de lignes du df
df.count()

13

In [6]:
#Tri du df par price et title
df.orderBy(["price", "title"], ascending=[1, 1]).show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|         Shakespeare| Signet  Classics| 7.95|Hamlet, Prince of...|        1603|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|             Marquez|Harper  Perennial| 14.0|One Hundred Years...|        1967|
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|       Rowling, J.K.|   Har

In [7]:
#Transformation du df en tableau de Row
print(df.collect())

[Row(author='Austen, Jane', edition='Penguin', price=18.2, title='Northanger Abbey', year_written=1814), Row(author='Tolstoy, Leo', edition='Penguin', price=12.7, title='War and Peace', year_written=1865), Row(author='Tolstoy, Leo', edition='Penguin', price=13.5, title='Anna Karenina', year_written=1875), Row(author='Woolf, Virginia', edition='Harcourt Brace', price=25.0, title='Mrs. Dalloway', year_written=1925), Row(author='Cunnningham, Michael', edition='Harcourt Brace', price=12.35, title='The Hours', year_written=1999), Row(author='Twain, Mark', edition='Penguin', price=5.76, title='Huckleberry Finn', year_written=1865), Row(author='Dickens, Charles', edition='Random House', price=5.75, title='Bleak House', year_written=1870), Row(author='Twain, Mark', edition='Random House', price=7.75, title='Tom Sawyer', year_written=1862), Row(author='Woolf, Virginia', edition='Penguin', price=29.0, title="A Room of One's Own", year_written=1922), Row(author='Rowling, J.K.', edition='Harcourt 

In [8]:
#Ajouter une ligne au df
df2 = df.collect()
df2.append({'author':'Abadie, Alex', 'edition':'koala', 'price':0.1, 'title':'Solutec', 'year_written':2023})
df = spark.createDataFrame(df2)
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

In [10]:
#Supprimer une ou plusieurs lignes au df
df = df.where(df.edition != 'koala')
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

In [18]:
#Grouper les valeurs du df
df_group = df.groupBy('edition').count()
df_group.show()

+-----------------+-----+
|          edition|count|
+-----------------+-----+
| Signet  Classics|    1|
|Harper  Perennial|    1|
|     Random House|    2|
|   Harcourt Brace|    3|
|          Penguin|    6|
+-----------------+-----+

